In [1]:
#dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

# cleaning crime rate starts here:

In [2]:
#setting file path and reading in the crime file
file = Path("../Resources/cleaner_crime_rate.csv")
df = pd.read_csv(file)
df.tail()

,County,Population,Violent,Murder,Rape,Robbery,Aggrevated assault,Property,Burglary,Vehicle theft,Larceny theft
54,Tuolumne,"54,532",357.6,3.7,124.7,23.8,205.4,"1,969.50",687.7,185.2,"1,096.60"
55,Ventura,"846,050",216.3,2.8,27.4,62.3,123.8,"1,493.30",230,145.9,"1,117.40"
56,Yolo,"220,896",288.4,2.7,39.4,79.7,166.6,"3,007.80",418.7,265.7,"2,323.30"
57,Yuba,"77,342",421.5,9.1,53.0,78.9,280.6,"2,384.20",523.6,614.2,"1,246.40"
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#pulling only the columns we want to look at
#note: violent includes murder, rape, robbery, aggevated assault and property includes burglary, vehicle theft,
#and larceny theft
minimized_df = df[["County","Population","Violent", "Property"]]
minimized_df.head()

,County,Population,Violent,Property
0,Alameda,"1,664,783",584.7,"4,008.40"
1,Alpine,"1,149","1,131.40","2,785.00"
2,Amador,"37,820",290.9,"1,771.50"
3,Butte,"221,521",462.3,"2,402.50"
4,Calaveras,"45,085",321.6,"1,858.70"


In [4]:
#checking the data types
minimized_df.dtypes

County        object
Population    object
Violent       object
Property      object
dtype: object

In [5]:
#removing the comma mapping for the numbers so that the values are floats
minimized_df.replace(',', '', regex=True, inplace=True)
minimized_df.head()

/var/folders/3_/pnzxh0yn5lqbg2gh85_bx9gc0000gn/T/ipykernel_7818/483355146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minimized_df.replace(',', '', regex=True, inplace=True)


,County,Population,Violent,Property
0,Alameda,1664783,584.7,4008.40
1,Alpine,1149,1131.40,2785.00
2,Amador,37820,290.9,1771.50
3,Butte,221521,462.3,2402.50
4,Calaveras,45085,321.6,1858.70


In [6]:
#changing the data types of the numeric columns
minimized_df[["Population", "Violent", "Property"]] = minimized_df[["Population", "Violent", "Property"]].astype({"Population": float, "Violent": float, "Property": float})

/var/folders/3_/pnzxh0yn5lqbg2gh85_bx9gc0000gn/T/ipykernel_7818/3034067415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minimized_df[["Population", "Violent", "Property"]] = minimized_df[["Population", "Violent", "Property"]].astype({"Population": float, "Violent": float, "Property": float})


In [7]:
#making sure the data types were changed correctly 
minimized_df.dtypes

County         object
Population    float64
Violent       float64
Property      float64
dtype: object

In [8]:
#sorted ascending so that it'll comply with plotly standards
sorted_df = minimized_df.sort_values("Population")
sorted_df.head()

,County,Population,Violent,Property
1,Alpine,1149.0,1131.4,2785.0
45,Sierra,3210.0,31.2,965.7
24,Modoc,9637.0,446.2,684.9
25,Mono,13517.0,495.7,1383.4
52,Trinity,13637.0,447.3,1737.9


In [9]:
reset_crime_index = sorted_df.reset_index(drop = True)
reset_crime_index

,County,Population,Violent,Property
0,Alpine,1149.0,1131.4,2785.0
1,Sierra,3210.0,31.2,965.7
2,Modoc,9637.0,446.2,684.9
3,Mono,13517.0,495.7,1383.4
4,Trinity,13637.0,447.3,1737.9
5,Mariposa,18066.0,437.3,1007.4
6,Plumas,18242.0,701.7,1721.3
7,Inyo,18572.0,506.1,2116.1
8,Colusa,21990.0,263.8,1578.0
9,Del Norte,27127.0,401.8,2963.8


In [10]:
cleaned_crime = reset_crime_index.sort_values("Violent")
cleaned_crime

,County,Population,Violent,Property
1,Sierra,3210.0,31.2,965.7
36,Placer,395978.0,150.8,1550.1
29,El Dorado,190018.0,170.0,1346.7
32,Marin,262240.0,196.8,2075.6
55,Orange,3192987.0,212.9,1853.6
45,Ventura,846050.0,216.3,1493.3
34,San Luis Obispo,278355.0,226.3,1839.0
44,San Mateo,774231.0,237.3,2101.7
16,San Benito,61513.0,239.0,767.3
21,Nevada,97820.0,243.3,1217.5


In [11]:
renamed_crime = cleaned_crime.rename(columns={"Violent": "Violent Crime Rate Per 100k", "Property": "Property Crime Rate Per 100k"})
renamed_crime.head()

,County,Population,Violent Crime Rate Per 100k,Property Crime Rate Per 100k
1,Sierra,3210.0,31.2,965.7
36,Placer,395978.0,150.8,1550.1
29,El Dorado,190018.0,170.0,1346.7
32,Marin,262240.0,196.8,2075.6
55,Orange,3192987.0,212.9,1853.6


In [12]:
resetting_index = renamed_crime.dropna (how = "any")
resetting_index

,County,Population,Violent Crime Rate Per 100k,Property Crime Rate Per 100k
1,Sierra,3210.0,31.2,965.7
36,Placer,395978.0,150.8,1550.1
29,El Dorado,190018.0,170.0,1346.7
32,Marin,262240.0,196.8,2075.6
55,Orange,3192987.0,212.9,1853.6
45,Ventura,846050.0,216.3,1493.3
34,San Luis Obispo,278355.0,226.3,1839.0
44,San Mateo,774231.0,237.3,2101.7
16,San Benito,61513.0,239.0,767.3
21,Nevada,97820.0,243.3,1217.5


In [13]:
#exporting dataframe as a CSV
resetting_index.to_csv("../Resources/crime_rate_v1.csv", index = False, header = True)

# cleaning homelessess starts here

In [14]:
#reading in homelessness file
file2 = Path("../Resources/experiencing_homelessness_ethnicity_demographics.csv")
homeless = pd.read_csv(file2)
homeless.head()

,CALENDAR_YEAR,LOCATION_ID,LOCATION,ETHNICITY,EXPERIENCING_HOMELESSNESS
0,2017,All,California,Hispanic/Latinx,60962
1,2017,All,California,Not Hispanic/Latinx,119153
2,2017,All,California,Unknown,3791
3,2017,CA-500,Santa Clara County CoC,Hispanic/Latinx,5034
4,2017,CA-500,Santa Clara County CoC,Not Hispanic/Latinx,5074


In [15]:
#filtering for only the year 2019
filtered_homeless = homeless[homeless["CALENDAR_YEAR"] == 2019]
filtered_homeless.head()

,CALENDAR_YEAR,LOCATION_ID,LOCATION,ETHNICITY,EXPERIENCING_HOMELESSNESS
256,2019,All,California,Hispanic/Latinx,81432
257,2019,All,California,Not Hispanic/Latinx,153135
258,2019,All,California,Unknown,5027
259,2019,CA-500,Santa Clara County CoC,Hispanic/Latinx,5972
260,2019,CA-500,Santa Clara County CoC,Not Hispanic/Latinx,5938


In [16]:
#removing where ethnicity is unknown
df1 = filtered_homeless.loc[(filtered_homeless["ETHNICITY"] != "Unknown")]
df1.head()

,CALENDAR_YEAR,LOCATION_ID,LOCATION,ETHNICITY,EXPERIENCING_HOMELESSNESS
256,2019,All,California,Hispanic/Latinx,81432
257,2019,All,California,Not Hispanic/Latinx,153135
259,2019,CA-500,Santa Clara County CoC,Hispanic/Latinx,5972
260,2019,CA-500,Santa Clara County CoC,Not Hispanic/Latinx,5938
262,2019,CA-501,San Francisco CoC,Hispanic/Latinx,2351


In [17]:
#removing data from California overall
df2 = df1.loc[(df1["LOCATION"] != "California")]
df2.head()

,CALENDAR_YEAR,LOCATION_ID,LOCATION,ETHNICITY,EXPERIENCING_HOMELESSNESS
259,2019,CA-500,Santa Clara County CoC,Hispanic/Latinx,5972
260,2019,CA-500,Santa Clara County CoC,Not Hispanic/Latinx,5938
262,2019,CA-501,San Francisco CoC,Hispanic/Latinx,2351
263,2019,CA-501,San Francisco CoC,Not Hispanic/Latinx,5429
265,2019,CA-502,Alameda County CoC,Hispanic/Latinx,1416


In [18]:
#removing the null values in the experiencing_homelessness column marked with a *
df3 = df2.loc[(df2["EXPERIENCING_HOMELESSNESS"] != "*")]
df3.head()

,CALENDAR_YEAR,LOCATION_ID,LOCATION,ETHNICITY,EXPERIENCING_HOMELESSNESS
259,2019,CA-500,Santa Clara County CoC,Hispanic/Latinx,5972
260,2019,CA-500,Santa Clara County CoC,Not Hispanic/Latinx,5938
262,2019,CA-501,San Francisco CoC,Hispanic/Latinx,2351
263,2019,CA-501,San Francisco CoC,Not Hispanic/Latinx,5429
265,2019,CA-502,Alameda County CoC,Hispanic/Latinx,1416


In [19]:
#exporting to a CSV
#note more changes needed to be made on this exported CSV in excel (making a total column)
df3.to_csv("../Resources/homeless_v1.csv", index = False, header = True)

In [20]:
file_4 = Path("../Resources/homeless_v1.csv")
file_4_df = pd.read_csv(file_4)
file_4_df.head()

,CALENDAR_YEAR,LOCATION_ID,LOCATION,ETHNICITY,EXPERIENCING_HOMELESSNESS
0,2019,CA-500,Santa Clara County CoC,Hispanic/Latinx,5972
1,2019,CA-500,Santa Clara County CoC,Not Hispanic/Latinx,5938
2,2019,CA-501,San Francisco CoC,Hispanic/Latinx,2351
3,2019,CA-501,San Francisco CoC,Not Hispanic/Latinx,5429
4,2019,CA-502,Alameda County CoC,Hispanic/Latinx,1416


In [21]:
file_4_df.dtypes

CALENDAR_YEAR                 int64
LOCATION_ID                  object
LOCATION                     object
ETHNICITY                    object
EXPERIENCING_HOMELESSNESS     int64
dtype: object

# cleaning unemployment starts here

In [22]:
file_5 = Path("../Resources/laborforceandunemployment_monthly_2024119.csv")
unemployment = pd.read_csv(file_5)
unemployment.head()

,Area Name,Area Type,Year,Month,Date_Numeric,Seasonally Adjusted(Y/N),Status,Labor Force,Employment,Unemployment,Unemployment Rate
0,California,State,1976,January,Jan-76,N,Final,9672362,8668016,1004346,10.4
1,California,State,1976,January,Jan-76,Y,Final,9774280,8875685,898595,9.2
2,Los Angeles County,County,1976,January,Jan-76,N,Final,3364151,3040058,324093,9.6
3,Los Angeles County,County,1976,January,Jan-76,Y,Final,3381856,3081806,300050,8.9
4,Los Angeles-Long Beach-Glendale MD,Metropolitan Area,1976,January,Jan-76,N,Final,3364151,3040058,324093,9.6


In [23]:
unemployment2 = unemployment.loc[(unemployment["Area Type"] == "County")]
unemployment2.head()

,Area Name,Area Type,Year,Month,Date_Numeric,Seasonally Adjusted(Y/N),Status,Labor Force,Employment,Unemployment,Unemployment Rate
2,Los Angeles County,County,1976,January,Jan-76,N,Final,3364151,3040058,324093,9.6
3,Los Angeles County,County,1976,January,Jan-76,Y,Final,3381856,3081806,300050,8.9
8,Los Angeles County,County,1976,February,Feb-76,N,Final,3364401,3050306,314095,9.3
9,Los Angeles County,County,1976,February,Feb-76,Y,Final,3381218,3081497,299721,8.9
14,Los Angeles County,County,1976,March,Mar-76,N,Final,3361820,3069797,292023,8.7


In [24]:
unemployment3 = unemployment2.loc[(unemployment2["Year"] == 2019)]
unemployment3

,Area Name,Area Type,Year,Month,Date_Numeric,Seasonally Adjusted(Y/N),Status,Labor Force,Employment,Unemployment,Unemployment Rate
133334,Alameda County,County,2019,January,Jan-19,N,Final,840900,811200,29800,3.5
133335,Alpine County,County,2019,January,Jan-19,N,Final,720,690,30,4.3
133336,Amador County,County,2019,January,Jan-19,N,Final,14570,13830,740,5.1
133337,Butte County,County,2019,January,Jan-19,N,Final,99200,92300,6900,6.9
133338,Calaveras County,County,2019,January,Jan-19,N,Final,20900,19860,1040,5.0
...,...,...,...,...,...,...,...,...,...,...,...
144641,Tulare County,County,2019,December,Dec-19,N,Final,199800,180000,19800,9.9
144642,Tuolumne County,County,2019,December,Dec-19,N,Final,20600,19730,870,4.2
144643,Ventura County,County,2019,December,Dec-19,N,Final,423800,408800,14900,3.5
144644,Yolo County,County,2019,December,Dec-19,N,Final,107800,103300,4400,4.1


In [25]:
unemployment4 = unemployment3.loc[(unemployment3["Month"] == "December")]
unemployment4.head()

,Area Name,Area Type,Year,Month,Date_Numeric,Seasonally Adjusted(Y/N),Status,Labor Force,Employment,Unemployment,Unemployment Rate
144587,Alameda County,County,2019,December,Dec-19,N,Final,845300,822800,22500,2.7
144588,Alpine County,County,2019,December,Dec-19,N,Final,520,500,30,4.8
144589,Amador County,County,2019,December,Dec-19,N,Final,14670,14100,570,3.9
144590,Butte County,County,2019,December,Dec-19,N,Final,97700,93200,4400,4.5
144591,Calaveras County,County,2019,December,Dec-19,N,Final,21430,20660,770,3.6


In [26]:
unemployment5 = unemployment4.loc[(unemployment4["Seasonally Adjusted(Y/N)"] == "N")]
unemployment5.head()

,Area Name,Area Type,Year,Month,Date_Numeric,Seasonally Adjusted(Y/N),Status,Labor Force,Employment,Unemployment,Unemployment Rate
144587,Alameda County,County,2019,December,Dec-19,N,Final,845300,822800,22500,2.7
144588,Alpine County,County,2019,December,Dec-19,N,Final,520,500,30,4.8
144589,Amador County,County,2019,December,Dec-19,N,Final,14670,14100,570,3.9
144590,Butte County,County,2019,December,Dec-19,N,Final,97700,93200,4400,4.5
144591,Calaveras County,County,2019,December,Dec-19,N,Final,21430,20660,770,3.6


In [27]:
sorted_unemployment = unemployment5.reset_index(drop = True)
sorted_unemployment.head()

,Area Name,Area Type,Year,Month,Date_Numeric,Seasonally Adjusted(Y/N),Status,Labor Force,Employment,Unemployment,Unemployment Rate
0,Alameda County,County,2019,December,Dec-19,N,Final,845300,822800,22500,2.7
1,Alpine County,County,2019,December,Dec-19,N,Final,520,500,30,4.8
2,Amador County,County,2019,December,Dec-19,N,Final,14670,14100,570,3.9
3,Butte County,County,2019,December,Dec-19,N,Final,97700,93200,4400,4.5
4,Calaveras County,County,2019,December,Dec-19,N,Final,21430,20660,770,3.6


In [28]:
renamed_unemployment = sorted_unemployment.rename(columns={"Area Name": "County"})
renamed_unemployment.head()

,County,Area Type,Year,Month,Date_Numeric,Seasonally Adjusted(Y/N),Status,Labor Force,Employment,Unemployment,Unemployment Rate
0,Alameda County,County,2019,December,Dec-19,N,Final,845300,822800,22500,2.7
1,Alpine County,County,2019,December,Dec-19,N,Final,520,500,30,4.8
2,Amador County,County,2019,December,Dec-19,N,Final,14670,14100,570,3.9
3,Butte County,County,2019,December,Dec-19,N,Final,97700,93200,4400,4.5
4,Calaveras County,County,2019,December,Dec-19,N,Final,21430,20660,770,3.6


In [29]:
minimized_unemployment = renamed_unemployment[["County", "Year", "Labor Force", "Employment", "Unemployment", "Unemployment Rate"]]
minimized_unemployment.head()

,County,Year,Labor Force,Employment,Unemployment,Unemployment Rate
0,Alameda County,2019,845300,822800,22500,2.7
1,Alpine County,2019,520,500,30,4.8
2,Amador County,2019,14670,14100,570,3.9
3,Butte County,2019,97700,93200,4400,4.5
4,Calaveras County,2019,21430,20660,770,3.6


In [30]:
minimized_unemployment.dtypes

County                object
Year                   int64
Labor Force            int64
Employment             int64
Unemployment           int64
Unemployment Rate    float64
dtype: object

In [31]:
sorted_unemployment = minimized_unemployment.sort_values("Unemployment Rate")
sorted_unemployment.head()

,County,Year,Labor Force,Employment,Unemployment,Unemployment Rate
40,San Mateo County,2019,460500,452000,8500,1.8
37,San Francisco County,2019,585100,573500,11700,2.0
20,Marin County,2019,138800,135900,2900,2.1
42,Santa Clara County,2019,1058000,1033900,24100,2.3
48,Sonoma County,2019,256600,250200,6400,2.5


In [32]:
sorted_unemployment.reset_index(drop = True)

,County,Year,Labor Force,Employment,Unemployment,Unemployment Rate
0,San Mateo County,2019,460500,452000,8500,1.8
1,San Francisco County,2019,585100,573500,11700,2.0
2,Marin County,2019,138800,135900,2900,2.1
3,Santa Clara County,2019,1058000,1033900,24100,2.3
4,Sonoma County,2019,256600,250200,6400,2.5
5,Orange County,2019,1618800,1577800,41000,2.5
6,Alameda County,2019,845300,822800,22500,2.7
7,San Luis Obispo County,2019,137500,133800,3700,2.7
8,Contra Costa County,2019,562200,546500,15800,2.8
9,Placer County,2019,189100,183800,5300,2.8


In [34]:
renamed_unemployment2 = sorted_unemployment.rename(columns={"Employment": "Employment Count", "Unemployment": "Unemployment Count"})
renamed_unemployment2.head()

,County,Year,Labor Force,Employment Count,Unemployment Count,Unemployment Rate
40,San Mateo County,2019,460500,452000,8500,1.8
37,San Francisco County,2019,585100,573500,11700,2.0
20,Marin County,2019,138800,135900,2900,2.1
42,Santa Clara County,2019,1058000,1033900,24100,2.3
48,Sonoma County,2019,256600,250200,6400,2.5


In [35]:
renamed_unemployment2.to_csv("../Resources/unemployment_v1.csv", index = False, header = True)